# BERT 기반의 한국어 Question Answering Model

본 과정은 Ainize Workspace에서 klue/bert-base 모델을 KLUE-MRC(Machine Reading Comprehension) 데이터 셋으로 학습/추론/평가하는 과정입니다.

해당 모델은 [Ainize Demo](https://main-klue-mrc-bert-scy6500.endpoint.ainize.ai/)에서 사용해보실 수 있고, [Ainize API](https://ainize.ai/scy6500/KLUE-MRC-BERT?branch=main)에서 API로 호출해보실 수 있습니다.

## 라이브러리

우선 본 과정에서 사용할 라이브러리들을 불러옵니다.

In [20]:
import json
import random

import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW

## 데이터 처리

KLUE 데이터 셋은 [KLUE Github](https://github.com/KLUE-benchmark/KLUE/tree/main/klue_benchmark/klue-mrc-v1)에서 확인하실 수 있으며, [HuggingFace KLUE datasets 라이브러리](https://huggingface.co/datasets/klue)를 통해서도 확인하실 수 있습니다.

우선 데이터를 살펴보겠습니다. 데이터는 train/dev(validation)으로 나누어져 있습니다.

데이터 안에는 context, context에 대한 질문, 답변, context 안에서 답변의 위치(answer_start) 등으로 이루어져 있는 걸 볼 수 있습니다.

In [2]:
with open("./klue_dataset/klue-mrc-v1_train.json", 'rb') as f:
    klue_dict = json.load(f)
klue_dict["data"][0]

{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'paragraphs': [{'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
   'qas': [{'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
     'answers': [{'text': '한 달가량', 'answer_start': 478},
      {'text': '한 달', 'answer_start': 478}],
     'question_type': 1,
     'is_impos

데이터 중에는 이처럼 'question_type': 3으로 처리되어 있는 데이터가 있는데 이는 context를 보고 답변 불가능한 데이터를 의미합니다.

답변 불가능한 데이터들은 answer가 빈 리스트로 존재하며 가짜 답"에 해당되는 plausible_answers 가 레이블링되어 있습니다.

In [3]:
klue_dict["data"][1]

{'title': '부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정',
 'paragraphs': [{'context': '부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 4개 과제가 선정되어 본격적인 사업 착수에 나선다. 과학기술정보통신부가 주관하는 ‘지역SW서비스사업화 지원사업’은 강소SW기업 및 초기 스타트업의 SW서비스 사업화 지원과 신시장 진출 지원을 통해 기업 경쟁력 강화와 지역경제 활성화를 도모하는 사업이다. 올해부터 2개년으로 진행되며, 국비와 시비, 민자 등 2년간 약 37억원의 예산이 투입된다. 앞서 진흥원은 부산의 미래 먹거리산업인 스마트해양, 지능형기계, 지능정보서비스 분야로 사전 수요조사를 진행했고, 평가를 통해 선정된 5개 과제를 공모사업에 신청했다. 그 결과 부산의 4개 과제가 최종 선정되는 쾌거를 거뒀다. 당 사업은 전국 진흥기관을 대상으로 공모를 시작해, 총 17개 지역에서 42개 과제가 선정되었으며, 4개 과제가 선정된 곳은 부산과 강원지역 뿐이다. 금번 선정된 과제들은 ‘인공지능융합센서와 서보 이송 로봇을 이용한 전단보강재의 자동용접시스템 개발’ 등 총 4개 과제다. 부산시가 지원하고, 부산정보산업진흥원과 지역기업, 대학, 연구소 등이 컨소시엄을 구성하여 기술개발 및 사업화 지원을 추진한다. 2개의 Track으로 구분되는 이번사업은 Track 1(SW중소기업)에서 ㈜에이아이플랫폼, 엔컴(주), Track 2(스타트업)에서는 ㈜토즈, 삼보테크놀로지를 지원한다. ○ ‘Track 1‘의 (주)에이아이플랫폼이 주관기업으로 진행하는 <인공지능 기반 망막 내 아밀로이드 플라크 영상 분석을 통한 치매조기진단 플랫폼 상용화>는 치매 확진의 원인이 되는 중요 단백질(아밀로이드 플라크)을 자체개발 관측장비로 진단한다. 이를 통해 치매를 조기 발견하여, 각종 경제적 비용과 치료 및 예방 등 사회적 문제를 해 결하고 시민들이 쉽게 접근 가능한 실효성 있는 치매관리

해당 데이터는 각 context에 대한 여러 question과 answer이 있는 형태이므로 이를 context, question, answer로 분리하겠습니다(한 context에 여러 질문, 답변이 있는 경우도 있기 때문에 context가 반복될 수 있습니다).

In [4]:
def read_klue(path):
    with open(path, 'rb') as f:
        klue_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in tqdm(klue_dict['data']):
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
  

    return contexts, questions, answers

현재 정답이 context에서 시작하는 위치(answer_start)만 있기 때문에 끝나는 위치도 구해보겠습니다.

In [5]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2

이제 데이터에 정답의 시작하는 위치, 끝나는 위치가 포함되었습니다. 

다음으로 토크나이저를 불러오겠습니다.

In [6]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

토큰화하는 방식은 BERT 논문의 Fine-tuning BERT on Different Tasks 부분을 참고하였습니다.

![Bert](https://i.imgur.com/Te9Fw87.png)

토크나이저에 train 데이터의 일부를 넣어 확인해보겠습니다.

input_ids를 보시면 위의 사진과 비슷하게 ```cls_token(id=2) context sep_token(id=3) question sep_token(id=3)```의 형식으로 토큰화된 것을 확인할 수 있습니다.

BERT 논문에서는 question, context의 순서로 입력하였지만, HuggingFace Docs에서는 context, question 순서로 입력하여 저는 context, question의 형태로 토큰화를 진행했습니다.

train 할 때의 데이터 순서와 inference 할 때의 데이터 순서만 맞춰주면 상관이 없습니다.

In [7]:
question = "북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?"
context = "올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다."
tokenizer(context, question)

{'input_ids': [2, 1446, 22555, 11477, 2116, 3932, 2210, 6530, 27135, 3670, 2367, 2062, 18, 3671, 886, 9775, 16311, 2073, 12982, 2178, 2062, 15513, 3309, 3681, 798, 2073, 5277, 1041, 2678, 11477, 2116, 3670, 2651, 4016, 28674, 18, 3932, 2210, 11945, 2170, 3881, 2460, 6530, 7831, 1060, 10526, 2170, 1513, 2259, 11477, 2165, 2020, 2079, 3979, 6233, 3814, 6530, 24028, 1116, 12468, 17552, 2170, 24902, 3802, 2178, 2116, 23772, 31369, 5844, 2170, 3911, 3569, 2170, 10760, 2205, 2259, 1039, 2073, 1187, 2116, 5740, 2062, 18, 4364, 2079, 11477, 2259, 18673, 2178, 2062, 22, 97, 23, 2210, 16, 3736, 2178, 4000, 4051, 5947, 3670, 2367, 2062, 18, 11477, 2259, 19880, 2062, 2219, 2470, 1174, 18956, 26797, 2145, 1891, 2398, 1322, 2399, 2470, 22152, 2128, 2292, 2097, 26797, 2052, 4026, 4605, 2496, 2259, 11477, 2165, 2020, 27135, 4848, 2259, 1187, 2138, 936, 4538, 18, 11477, 2165, 2020, 2073, 3801, 2210, 6530, 1060, 7831, 7755, 6233, 12314, 4795, 3619, 2210, 2678, 3690, 25848, 2097, 4997, 18787, 2299, 2118,

이제 데이터들을 토큰화 해보겠습니다. 

truncation은 모델에 입력되는 길이가 모델에 허용되는 최대 입력 길이보다 클 경우 모델의 최대 입력 길이로 자르는 역할을 합니다.

반면 padding은 모델에 입력되는 길이가 모델에 허용되는 최대 입력 길이보다 작을 경우 모델의 최대 입력 길이까지 채우는 역할을 합니다.

토큰화가 다 진행되었다면 다음으로 answer의 시작/종료 위치를 토큰화된 context 안에서 answer의 시작/종료 위치로 바꿔야 합니다. 

이를 위해 ```char_to_token```을 사용하였는데, 이는 원래의 문자열에서 토큰의 인덱스를 가져오는 역할을 합니다.

In [8]:
class KlueDataset(Dataset):
    def __init__(self, contexts, questions, answers, model_max_position_embedings, tokenizer):
        self.tokenizer = tokenizer
        self.answers = answers
        self.questions = questions
        self.contexts = contexts
        self.model_max_position_embedings = model_max_position_embedings
        print("Tokenizing ...")
        self.encodings = self.tokenizer(self.contexts, 
                                        self.questions,
                                        max_length=512,
                                        truncation=True,
                                        padding="max_length",
                                        return_token_type_ids=False)
        print("Done !!!")
        self.add_token_positions()
        
    def add_token_positions(self):
        start_positions = []
        end_positions = []
        for i in range(len(self.answers)):
            start_positions.append(self.encodings.char_to_token(i, self.answers[i]['answer_start']))
            end_positions.append(self.encodings.char_to_token(i, self.answers[i]['answer_end'] - 1))

            # positions 값이 None 값이라면, answer가 포함된 context가 잘렸다는 의미
            if start_positions[-1] is None:
                start_positions[-1] = self.model_max_position_embedings
            if end_positions[-1] is None:
                end_positions[-1] = self.model_max_position_embedings

        self.encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

        
    def get_data(self):
        return {"contexts":self.contexts, 'questions':self.questions, 'answers':self.answers}
    
    
    def get_encodings(self):
        return self.encodings
        
    
    def __getitem__(self, idx):
        return {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    
    def __len__(self):
        return len(self.encodings['input_ids'])

In [9]:
contexts, questions, answers = read_klue("./klue_dataset/klue-mrc-v1_train.json")
add_end_idx(answers, contexts)
train_dataset = KlueDataset(contexts, questions, answers, 512, tokenizer)

  0%|          | 0/12174 [00:00<?, ?it/s]

Tokenizing ...
Done !!!


## 모델 학습

우선 학습에 사용할 모델을 불러오겠습니다.

모델을 불러오면 경고가 발생하는 데 이는 BertForQuestionAnswering를 fine-tuning 하기 전에는 모델의 성능이 좋지 않을 것이라고 알려줍니다.

In [10]:
model = AutoModelForQuestionAnswering.from_pretrained("klue/bert-base")

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

학습에 필요한 파라미터들을 정의해줍니다.

In [11]:
EPOCH = 3
LEARNING_RATE = 5e-5
BATCH_SIZE = 8

학습할 때는 모델에 context+question을 인코딩한 input_ids, attention_mask, start_positions, end_positions가 input으로 들어갑니다.

In [12]:
def train_runner(model, dataset, batch_size, num_train_epochs, learning_rate):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    model.to(device)
    model.train()
    train_dataloader = DataLoader(dataset=dataset, batch_size=batch_size)
    global_total_step = len(train_dataloader) * num_train_epochs
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0)
    print("TRAIN START")
    with tqdm(total=global_total_step, unit='step') as t:
        total = 0
        total_loss = 0
        for epoch in range(num_train_epochs):
            for batch in train_dataloader:
                optimizer.zero_grad()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                start_positions = batch['start_positions'].to(device)
                end_positions = batch['end_positions'].to(device)
                outputs = model(input_ids,
                             attention_mask=attention_mask,
                             start_positions=start_positions,
                             end_positions=end_positions)
                loss = outputs.loss
                loss.backward()
                optimizer.step()
                
                batch_loss = loss.item() * len(input_ids)
                total += len(input_ids)
                total_loss += batch_loss
                global_total_step += 1
                t.set_postfix(loss="{:.6f}".format(total_loss / total), batch_loss="{:.6f}".format(batch_loss))
                t.update(1)
                
                del input_ids
                del attention_mask
                del start_positions
                del end_positions
                del outputs
                del loss
    model.save_pretrained("./klue_output_model")
    print("TRAIN END")

학습을 진행합니다. (V100에서 3에폭 기준 약 30분정도 소요 됐습니다.)

In [13]:
train_runner(model,train_dataset, BATCH_SIZE, EPOCH, LEARNING_RATE)

TRAIN START


  0%|          | 0/6624 [00:00<?, ?step/s]

TRAIN END


## 모델 평가

dev 데이터를 가지고 학습된 모델을 평가해보겠습니다.

dev 데이터는 한 질문에 여러 답변이 있어도 각각 나눌 필요가 없어 원래 사용하던 코드를 조금 수정했습니다.

In [14]:
def read_dev_klue(path):
    with open(path, 'rb') as f:
        klue_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in tqdm(klue_dict['data']):
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                temp_answer = []
                for answer in qa['answers']:
                    temp_answer.append(answer['text'])
                if len(temp_answer) != 0: # answers의 길이가 0 == 답변할 수 없는 질문
                    contexts.append(context)
                    questions.append(question)
                    answers.append(temp_answer)

    return contexts, questions, answers

In [15]:
dev_contexts, dev_questions, dev_answers = read_dev_klue("./klue_dataset/klue-mrc-v1_dev.json")

  0%|          | 0/5075 [00:00<?, ?it/s]

우선 모델에 dev 데이터를 넣어 예측값을 얻어보겠습니다.

모델의 input 값으로는 input_ids, attention_mask가 들어가고, output으로는 logits 값들이 나옵니다.

logits 값 들은 입력으로 들어온 input_ids에서 각 토큰들이 정답이 될 확률을 나타냅니다.

이 확률값들의 최대값을 찾으면 그 토큰이 가장 가능성이 높은 답변의 시작점/끝점이 됩니다.

이 토큰들을 가지고 디코딩을 거치면 문자 형태의 예측값이 나오게 됩니다.

In [16]:
def prediction(contexts, questions):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    model.to(device)
    model.eval()
    
    result = []
    
    with torch.no_grad():
        for context, question in zip(contexts, questions):
            encodings = tokenizer(context, question, max_length=512, truncation=True,
                                     padding="max_length", return_token_type_ids=False)
            encodings = {key: torch.tensor([val]) for key, val in encodings.items()}
            
            input_ids = encodings["input_ids"].to(device)
            attention_mask = encodings["attention_mask"].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            start_logits, end_logits = outputs.start_logits, outputs.end_logits
            token_start_index, token_end_index = start_logits.argmax(dim=-1), end_logits.argmax(dim=-1)
            pred_ids = input_ids[0][token_start_index: token_end_index + 1]
            pred = tokenizer.decode(pred_ids)
            result.append(pred)

    return result

In [17]:
pred_answers = prediction(dev_contexts, dev_questions)
pred_answers

['뉴 530i 25주년 에디션 ( 25대 한정 ) 은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션',
 '독일 뒤셀도르프로',
 '링크트인',
 '페이스북',
 '마드리드',
 '국제 원자재값 하락',
 '5조원',
 '‘ 운영허가 인증서',
 '1년',
 '노르웨이',
 '16종',
 '필리핀',
 '김교성',
 '79달러',
 '존 위클리프',
 '존 위클리프',
 '지기스문트',
 '복정역',
 '내일',
 '한국예술종합학교 음악원',
 '로사다운자켓 ’',
 '키프로스',
 '피렌체',
 '보석',
 '시칠리아의 [UNK] 왕',
 '부레',
 '프리오노수쿠스',
 '피부',
 '척추',
 '분류학적 접근',
 '진양서류의 부서지기 쉬운 뼈',
 '손병석',
 '충주 2차 푸르지오',
 '공직자윤리법',
 '8월 20일',
 '가상 색상 "',
 '2년6개월',
 '9월22일',
 '소련 헌법',
 '인도',
 'Uraniborg',
 '프톨레마이우스',
 '프라하',
 '히파르코스',
 '아베크 변주곡',
 '577',
 '각막 궤양이',
 '',
 '2000만원',
 '개혁파',
 '조개류',
 '아드레날린',
 '6일',
 '레이다',
 '미노프스키 입자',
 '도로스',
 '연방군 수뇌',
 '샤아 소좌',
 '쿠니스',
 '',
 '안덕수',
 '식품위생법',
 '스타벅스',
 '6개',
 '5억원',
 '2004년',
 '《 대전망 2014 》',
 '8년',
 '민간인',
 '10개',
 '',
 '데프레닐',
 '미용',
 '고려은단',
 '쿠에농강',
 '',
 '제약협회',
 '도조',
 '등산 스틱',
 '1만7567명',
 '이달 17일',
 '오사카 츠타야 에비스바시',
 '13시간 35분',
 '3000억원',
 '서울지방변호사회',
 '큰 돈',
 '강승훈',
 '울산',
 '19단계',
 '자신들을 탄압한 사람들을 보복하고 디오게네스 등을 살해했다. 이에 반바리사이파는

이제 예측한 답과 실제 답을 비교하여 모델의 성능을 평가해보겠습니다.

진행할 평가는 EM(exact match)으로 QA 작업에 가장 일반적으로 사용되는 측정 항목입니다. 

예측한 답과 실제 답이 정확히 일치하면 점수를 얻게 됩니다

In [18]:
def em_evalutate(prediction_answers, real_answers):
    total = len(prediction_answers)
    exact_match = 0
    for prediction_answer, real_answer in zip(prediction_answers, real_answers):
        if prediction_answer in real_answer:
            exact_match += 1
    
    return (exact_match/total) * 100

In [19]:
em_score = em_evalutate(pred_answers, dev_answers)
em_score

50.52395209580839

지금까지 klue/bert-base 모델을 KLUE-MRC(Machine Reading Comprehension) 데이터 셋으로 학습/추론/평가하는 과정을 진행해보았습니다.

해당 과정이 도움이 되셨으면 좋겠습니다.

이 모델은 [Ainize HuggingFace](https://huggingface.co/ainize/klue-bert-base-mrc)에서도 확인하실 수 있습니다.

## Reference

1. [Huggingface Fine-tuning with custom datasets](https://huggingface.co/transformers/custom_datasets.html#qa-squad)
2. [KLUE Github](https://github.com/KLUE-benchmark/KLUE)